<a href="https://colab.research.google.com/github/andersonfedel/DATASUS-SIA-Extract/blob/main/DATASUS_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Montagem do Drive Google 
from google.colab import drive 
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
#Instalação das bibliotecas necessárias
!pip install pysus --no-deps
!pip install dbfread
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 5.2 MB/s 
  Created wheel for pysus: filename=PySUS-0.5.16-cp37-cp37m-linux_x86_64.whl size=1443549 sha256=05ea17b8cb0c856378681a7a603519bff70d947d83f332fe2fa1b264a0b3a4d7
  Stored in directory: /root/.cache/pip/wheels/44/38/53/46e43fb114fef97da0eb2d2985a9bfe6874e74f808ca9f4e30
Successfully built pysus
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pysus 0.5.16 requires datashader==0.13.0, which is not installed.
pysus 0.5.16 requires elasticsearch==8.1.2, which is not installed.
pysus 0.5.16 requires facets-overview==1.0.0, which is not installed.
pysus 0.5.16 requires fastparquet==0.8.1, which is not installed.
pysus 

In [3]:
from ftplib import FTP
import numpy as np
import pandas as pd
import os
from pysus.online_data.SIA import download, show_datatypes
from pysus.utilities.readdbc import read_dbc, read_dbc_dbf, dbc2dbf, dbf_to_csvgz
import sys, errno
import time

CACHEPATH = os.getcwd() + "/" #variável com local dos arquivos temporários

### Nessa fase baixaremos os arquivos direto do FTP DATASUS

In [4]:
# Configurando endereço FTP dos arquivos SIA DATASUS. 
# Os arquivos baixados ficarão armazenados de forma temporária em CATHEPATH .
# No SIA os arquivos seguem o formato PAUFAAMM.DBC ou PAUFAAMMa.DBC no caso de São Paulo 

ftp = FTP('ftp.datasus.gov.br')
ftp.login()
ftp.cwd('/dissemin/publicos/SIASUS/200801_/Dados')

dir_list = []#Aqui ficarão guardados todos os arquivos disponíveis para download
ftp.dir(dir_list.append)

### A partir daqui os arquivos .DBC baixados ficarão armazenados em uma pasta temporária. Serão apagados assim que finalizada a sessão do Google Colab.

In [12]:
#Pode-se trabalhar as variáveis 'local', 'ano' e 'mes' para baixar os arquivos desejados.
#Exemplo de iteração para baixar vários arquivos:

locais = ["PB", "RN"]
anos = ["21"]
meses = ["01"]

arquivos = []

for local in locais:
  for ano in anos:
    for mes in meses:
      #filename = f"PA{local}{ano}{mes}.dbc"
      tempfilename = f"PA{local}{ano}{mes}"
      for i in dir_list:
        if tempfilename in i:
          #print(i.split()[-1])
          arquivos.append(i.split()[-1])
      for arquivo in arquivos:
        A = arquivo
        try:
          ftp.retrbinary("RETR " + arquivo ,open(CACHEPATH + A, 'wb').write)
        except IOError as e:
          if e.errno == errno.EPIPE:
            pass

### A próxima célula converterá todos os arquivos .DBC baixados até agora para o formato .parquet, em pastas divididas por estado no Google Drive montado, no formato .../Projeto_DATASUS/UF. 
###Por exemplo: o arquivo do mês de janeiro de 2021 do Acre será armazendo como: .../Projeto_DATASUS/AC/PAAC2101.parquet

In [13]:
#É preciso Google Drive já montado para rodar essas etapas

#Verificação e criação da pasta do projeto 
if not os.path.isdir("/content/gdrive/MyDrive/Projeto_DATASUS"):
  os.mkdir("/content/gdrive/MyDrive/Projeto_DATASUS")

dirPATH = "/content/gdrive/MyDrive/Projeto_DATASUS"

filelist = [] #Lista temporária com registro dos arquivos processados

for file in os.listdir(CACHEPATH): #Iterando sobre arquivos baixados anteriormente
  if str(file).endswith(".dbc"):   
    if len(file) == 12: #Capturando o nome do arquivo no formato PAUFAAMM
      filename = file[:8] 
    else:
      filename = file[:9] 
    if os.path.isdir(dirPATH + "/" + file[2:4]): #Verificação e criação das pastas divididas por estado
      pass
    else: 
      os.mkdir(dirPATH + "/" + file[2:4])

    if filename + ".parquet" not in os.listdir(dirPATH + "/" + file[2:4]): #Checando se o arquivo .parquet ainda não foi criado
      dir = dirPATH + "/" + file[2:4]
      print(f"Criando arquivo {filename}"+".parquet")
      
      #Conversão de .DBC para pandas DataFrame
      dbc2dbf(CACHEPATH + filename + ".dbc",
              os.path.join(str(CACHEPATH + filename) + str(".dbf"))) 
      dbf_to_csvgz(os.path.join(str(CACHEPATH + filename)+ str(".dbf")))
      fn = os.path.join(CACHEPATH + filename + str(".csv.gz"))
      df = pd.read_csv(fn, parse_dates = ['PA_CMP'] )

      #Tratando coluna data de atendimento (conversão para dtype datetime)
      df["PA_CMP"] = df["PA_CMP"].apply(lambda x: x[:4]+"-"+x[4:6]+"-"+"01")
      df["PA_CMP"] = df["PA_CMP"].astype("datetime64[ns]")
      
      #Salvando o arquivo no formato .parquet em diretórios divididos por estado 
      df.astype({'PA_CIDSEC': str, 'PA_CIDCAS': str, 'PA_ETNIA': str,'PA_AUTORIZ':str, 'PA_CBOCOD':str }).to_parquet(dir + "/" + filename + ".parquet") 
      filelist.append(filename)

      #Deletando arquivos temporários para liberação de espaço em disco
      os.remove(str(CACHEPATH) +"/"+ str(filename) + str(".csv.gz"))
      os.remove(str(CACHEPATH) +"/"+ str(filename) + str(".dbf"))
    else:
      dir = dirPATH + "/" + file[2:4]
      print(f"O arquivo {filename}.parquet parece já ter sido criado. Confira nas pastas")

print("Concluído!")
print(f"Arquivos processados: {filelist}")

Criando arquivo PARN2101.parquet


Converting: 282231it [00:50, 5564.77it/s]
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20,22,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


O arquivo PAPB2101.parquet parece já ter sido criado. Confira nas pastas
Concluído!
Arquivos processados: ['PARN2101']


In [14]:
#Abrindo um arquivo. parquet pelo Pandas e verificando integridade dos dados
dfteste = pd.read_parquet(dir + "/" + filename + ".parquet")
dfteste

,PA_CODUNI,PA_GESTAO,PA_CONDIC,PA_UFMUN,PA_REGCT,PA_INCOUT,PA_INCURG,PA_TPUPS,PA_TIPPRE,PA_MN_IND,...,PA_CODOCO,PA_FLQT,PA_FLER,PA_ETNIA,PA_VL_CF,PA_VL_CL,PA_VL_INC,PA_SRV_C,PA_INE,PA_NAT_JUR
0,3095576,250050,PG,250050,0,0,0,2,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,118044.0,1244
1,3095576,250050,PG,250050,0,0,0,2,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,118044.0,1244
2,7366655,250730,PG,250730,0,0,0,36,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1244
3,7366655,250730,PG,250730,0,0,0,36,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1244
4,2399741,250750,PG,250750,0,0,0,7,0,I,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,3069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461002,4016017,250000,EP,250750,0,0,0,43,0,M,...,1,K,0,nan,0.0,0.0,0.0,125001.0,NaN,1023
461003,4016017,250000,EP,250750,0,0,0,43,0,M,...,1,K,0,nan,0.0,0.0,0.0,125001.0,NaN,1023
461004,4016017,250000,EP,250750,0,0,0,43,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1023
461005,4016017,250000,EP,250750,0,0,0,43,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1023
